In [1]:
import ollama 
import pandas as pd
import csv



In [2]:
import os
print(os.listdir())


['benchmark_existing_data.ipynb', 'benchmark_ourDatasets.ipynb', 'data', 'ourDatasets']


## evaluation for my_generated_mcq with llama3.1 and mistral

In [30]:
mcq = pd.read_csv('ourDatasets/my_generated_mcq.tsv', sep="\t")


im going to generate answer for the evaluation with llama 3.1

In [ ]:
def llm_answer(prompt):
   return  ollama.generate(model='llama3.1', prompt=prompt)['response']

new_data = dict()
new_data['answer'] = list()

limit = 2000

for index in range(limit):
   prompt = mcq.iloc[index]['Prompt']
   answer = llm_answer(prompt)
   print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(mcq.index)[:limit])
old_df = mcq.head(limit)

final_mcq = pd.merge(old_df, new_df, left_index=True, right_index=True)

print(final_mcq.head())

final_mcq.to_csv('ourDatasets/responses/mcq/final_mcq.csv', index=False)


im going to generate answer with mistral for the evaluation 

In [32]:
def llm_answer(prompt):
   return  ollama.generate(model='mistral', prompt=prompt)['response']

new_data = dict()
new_data['answer'] = list()

limit = 2000

for index in range(limit):
   prompt = mcq.iloc[index]['Prompt']
   answer = llm_answer(prompt)
   print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(mcq.index)[:limit])
old_df = mcq.head(limit)

final_mcq_mis = pd.merge(old_df, new_df, left_index=True, right_index=True)

print(final_mcq_mis.head())

final_mcq_mis.to_csv('ourDatasets/responses/mcq/final_mcq_mis.csv', index=False)

 A
 A
 A
 D
 A
 C
 C
 A
 A
 A
 A
 C
 A
 A
 A
 D
 B
 A
 D
 B
 A
 D
 A
 A
 A
 A
 C
 D
 A
 B
 A
 A
 A
 D
 A
 A
 D
 D
 C
 A
 A
 D
 C
 A
 D
 A
 A
 A
 A
 C
 A
 A
 D
 B
 A
 A
 A
 D
 C
 A
 B
 A
 A
 D
 B
 A
 D
 A
 B
 D
 A
 A
 A
 A
 A
 D
 D
 A
 C
 B
 A
 A
 A
 A

Explanation: The given question asks about the main purpose of gathering information about an organization's key employees and their roles. The correct option (A) specifies 'To compromise accounts', which is a common goal when attacking or targeting organizations, as it allows an attacker to gain unauthorized access to sensitive data through those employees' credentials. Options B, C, and D do not directly refer to the main purpose of gathering employee information in this context, although they could be potential uses depending on the specific circumstances.
 A) Phishing for Information
 A
 D
 B
 A
 B
 D
 B
 C
 B
 A
 A
 B
 C
 D
 D
 A
 A
 A
 A
 D) DS0030
 B
 B
 D
 B
 A
 B
 A
 C
 D
 D
 B
 B
 A
 A
 C
 A
 A
 A
 C
 D
 A
 A
 B
 A
 C
 A
 D
 B


now the evaluation part 

In [33]:
import pandas as pd

df1 = pd.read_csv("ourDatasets/responses/mcq/final_mcq.csv")
df2 = pd.read_csv('ourDatasets/responses/mcq/final_mcq_mis.csv')

def extract_predicted_option(s:str):
    options = ['A', 'B', 'C', 'D']
    s = s[::-1] #reverse string, find first uppercase with previous and next character not alphabetic (A*,  A , A.) ...
    for i in range(len(s)):
        if  s[i] in options and (True if i == 0 else not s[i-1].isalpha()) and (True if i == len(s)-1 else not s[i+1].isalpha()): 
            return s[i]
    return ""

def evaluate(df):
    correct_predictions = 0
    for i in range(len(df)):
        gt = df.iloc[i]['GT']
        llm_answer = df.iloc[i]['answer']
        predicted =  extract_predicted_option(llm_answer)
        if predicted == gt:
            correct_predictions+=1
    total_predictions = len(df)
    return correct_predictions / total_predictions * 100


print(evaluate(df1))
print(evaluate(df2))

40.949999999999996
44.45


## evaluation for my_generated_rcm with llama3.1 and mistral

In [35]:
rcm = pd.read_csv('ourDatasets/my_rcm_generated.tsv', sep="\t")


In [ ]:
def llm_answer(prompt):
   return ollama.generate(model="llama3.1", prompt=prompt, options={"temperature": 0}).get('response', "")


new_data = dict()
new_data['answer'] = list()
start = 0
limit = 150

for index in range(start, limit):
   prompt = rcm.iloc[index]['Prompt']
   print(index)
   answer = llm_answer(prompt)
   #print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(rcm.index)[:limit])
old_df = rcm.head(limit)

final_rcmllama = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())

final_rcmllama.to_csv('ourdatasets/responses/rcm/final_rcmllama.csv', index=False)

In [ ]:
def llm_answer(prompt):
   return ollama.generate(model="mistral", prompt=prompt, options={"temperature": 0}).get('response', "")


new_data = dict()
new_data['answer'] = list()
start = 0
limit = 150

for index in range(start, limit):
   prompt = rcm.iloc[index]['Prompt']
   print(index)
   answer = llm_answer(prompt)
   #print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(rcm.index)[:limit])
old_df = rcm.head(limit)

final_rcmllama = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())

final_rcmllama.to_csv('ourdatasets/responses/rcm/final_rcmM.csv', index=False)

In [36]:
import re
def extract_cwe_code(answer):
    if "CWE-OTHER" in answer.upper():
        return "NVD-CWE-Other"
    # Regular expression pattern
    pattern = r'CWE-\d+'
    # Extract the matching substrings
    matches = re.findall(pattern, answer)
    return matches[0] if len(matches) > 0 else "" 

In [37]:
test_df  = pd.read_csv('ourdatasets/responses/rcm/final_rcmllama.csv')
test_df.head()
for i in range(len(test_df)):
    print(test_df.iloc[i]["answer"])
    answer = test_df.iloc[i]["answer"]
    code = extract_cwe_code(answer)
    
    print("-"*10)
    print(code)
    print("*"*20)

Based on the CVE description, I would map this vulnerability to CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer.

Justification:

* The vulnerability is described as an "Out-of-bounds Write" issue, which indicates that the attacker can write data beyond the intended buffer boundaries.
* This is further confirmed by the statement that the issue is caused by using an insecure function allowing an attacker to overwrite arbitrary memory.
* The impact of this vulnerability includes Denial of Service (DoS), Remote Code Execution (RCE), and obtaining root privileges on the device, which suggests a significant security risk.

CWE-119
----------
CWE-119
********************
Based on the provided CVE description, I would map it to CWE-23 (Relative Path Traversal).

Justification:

The CVE description mentions "Relative Path Traversal" which is a specific type of vulnerability that occurs when an application allows an attacker to access files or directories outsid

In [38]:
#extract code 

new_data = dict()
new_data['answer'] = list()
start = 0
limit = 150

test_df  = pd.read_csv('ourdatasets/responses/rcm/final_rcmllama.csv')
test_df.head()
new_data = {'code':[]}
for i in range(len(test_df)):
    answer = test_df.iloc[i]["answer"]
    code = extract_cwe_code(answer)
    new_data['code'].append(code)



new_df = pd.DataFrame(new_data, index = list(rcm.index)[:limit])
old_df = rcm.head(limit)

final_rcm2 = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())

final_rcm2.to_csv('ourdatasets/responses/rcm/final_rcm_extracted_codelama.csv', index=False)
#evaluate rcm
correct_answers = 0
for i in range(150):
    if final_rcm2.iloc[i]['code'] == rcm.iloc[i]['GT']:
        correct_answers += 1
accuracy = correct_answers / len(final_rcm2)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 58.67%


In [39]:
#extract code 

new_data = dict()
new_data['answer'] = list()
start = 0
limit = 175

test_df  = pd.read_csv('ourdatasets/reponses/rcm/final_rcmM.csv')
test_df.head()
new_data = {'code':[]}
for i in range(len(test_df)):
    answer = test_df.iloc[i]["answer"]
    code = extract_cwe_code(answer)
    new_data['code'].append(code)



new_df = pd.DataFrame(new_data, index = list(rcm.index)[:limit])
old_df = rcm.head(limit)

final_rcm2 = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())

final_rcm2.to_csv('ourdatasets/responses/rcm/final_rcm_extracted_codeM.csv', index=False)
#evaluate rcm
correct_answers = 0
for i in range(175):
    if final_rcm2.iloc[i]['code'] == rcm.iloc[i]['GT']:
        correct_answers += 1
accuracy = correct_answers / len(final_rcm2)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 50.29%


## evaluation for my_generated_vsp with llama3.1 and mistral

In [3]:
vsp = pd.read_csv('ourDatasets/my_vsp_generated.tsv', sep = '\t')

In [ ]:
#function to generate model answer for vsp (change limit)
def llm_answer(prompt):
   return ollama.generate(model="llama3.1", prompt=prompt, options={"temperature": 0}).get('response', "")


new_data = dict()
new_data['answer'] = list()
start = 0
limit = 100

for index in range(start, limit):
   prompt = vsp.iloc[index]['Prompt']
   print(index)
   answer = llm_answer(prompt)
   #print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(vsp.index)[:limit])
old_df = vsp.head(limit)
final_vsplama = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())
final_vsplama.to_csv('ourDatasets/responses/vsp/final_vsplama.csv', index=False)

In [ ]:
#function to generate model answer for vsp (change limit)
def llm_answer(prompt):
   return ollama.generate(model="mistral", prompt=prompt, options={"temperature": 0}).get('response', "")


new_data = dict()
new_data['answer'] = list()
start = 0
limit = 100

for index in range(start, limit):
   prompt = vsp.iloc[index]['Prompt']
   print(index)
   answer = llm_answer(prompt)
   #print(answer)
   new_data['answer'].append(answer)


new_df = pd.DataFrame(new_data, index = list(vsp.index)[:limit])
old_df = vsp.head(limit)
final_vspM = pd.merge(old_df, new_df, left_index=True, right_index=True)
#print(final_rcm.head())
final_vspM.to_csv('ourDatasets/responses/vsp/final_vspM.csv', index=False)

0
1
2


In [4]:

# Load the CSV file
final_vsp = pd.read_csv('ourDatasets/responses/vsp/final_vspM.csv')

# Example CVSS vector to extract
example = "CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H"

# Initialize an empty dictionary to store the new data
new_data = dict()
new_data['vector'] = list()

# Define limit to control how many rows to process
limit = 150  # Modify the limit as needed

# Loop through each row in the DataFrame
for i in range(len(final_vsp)):
    ans = final_vsp.iloc[i]['answer']
    index = ans.find("CVSS:3.1")  # Find the starting index of the CVSS vector
    if index != -1:  # Ensure the vector is found
        vector = ans[index:index+len(example)]  # Extract the vector
    else:
        vector = None  # Handle cases where no CVSS vector is found
    new_data['vector'].append(vector)

# Create a new DataFrame for the extracted vectors
new_df = pd.DataFrame(new_data, index=final_vsp.index[:limit])

# Select the old data (first 'limit' rows)
old_df = final_vsp.head(limit)

# Merge the old data with the new CVSS vectors
final_vspmistral = pd.merge(old_df, new_df, left_index=True, right_index=True)

# Save the merged DataFrame to a new CSV file
final_vspmistral.to_csv('ourDatasets/responses/vsp/final_vspM_Vector.csv', index=False)

print("CSV file 'final_vspM_Vector.csv' has been created.")


CSV file 'final_vspM_Vector.csv' has been created.


In [4]:
!pip install cvss
from cvss import CVSS3

In [8]:
def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    cvss_score = c.scores()[0]
    return cvss_score

def compute_vsp_mad(fname, col):
    cvss_prefix = ''   # should be empty string if the model responds with the prefix
    #df = pd.read_csv(fname, sep='\t')
    df = final_vspmistral
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_vector = cvss_prefix + pred
            pred_score = get_cvss_score(pred_vector)
            gt_score = get_cvss_score(gt)
            print(pred_score, gt_score)
            print(abs(pred_score-gt_score))
            error += abs(pred_score-gt_score)
        except Exception as e:
            print('Invalid response at row {}'.format(idx+1))
            print(e)
            continue
        total += 1
    print(error)
    return error/total

In [9]:
compute_vsp_mad("", col = "vector")

8.4 7.8
0.6000000000000005
9.1 8.4
0.6999999999999993
Invalid response at row 3
Unknown value "A" in field "PR:A"
8.8 6.1
2.700000000000001
10.0 8.8
1.1999999999999993
8.2 5.5
2.6999999999999993
9.0 9.8
0.8000000000000007
8.8 8.1
0.7000000000000011
8.0 7.8
0.20000000000000018
10.0 9.8
0.1999999999999993
8.4 5.5
2.9000000000000004
10.0 4.6
5.4
8.6 7.5
1.0999999999999996
6.4 8.8
2.4000000000000004
9.1 8.8
0.29999999999999893
9.6 7.1
2.5
8.8 5.4
3.4000000000000004
7.7 7.8
0.09999999999999964
9.1 5.5
3.5999999999999996
9.8 8.8
1.0
9.8 8.8
1.0
8.0 6.5
1.5
9.6 8.8
0.7999999999999989
7.6 7.8
0.20000000000000018
7.6 7.8
0.20000000000000018
10.0 5.5
4.5
10.0 7.5
2.5
8.6 4.3
4.3
9.9 5.4
4.5
10.0 6.5
3.5
8.0 7.8
0.20000000000000018
9.0 7.1
1.9000000000000004
7.5 5.5
2.0
10.0 5.4
4.6
9.6 6.5
3.0999999999999996
4.9 6.3
1.3999999999999995
7.2 7.0
0.20000000000000018
8.8 6.1
2.700000000000001
9.0 5.3
3.7
8.4 7.1
1.3000000000000007
10.0 6.5
3.5
10.0 9.8
0.1999999999999993
9.6 6.1
3.5
9.1 6.2
2.8999999

2.4162162162162164

evaluation for vsp lama 

In [11]:

# Load the CSV file
final_vsp = pd.read_csv('ourDatasets/responses/vsp/final_vsplama.csv')

# Example CVSS vector to extract
example = "CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H"

# Initialize an empty dictionary to store the new data
new_data = dict()
new_data['vector'] = list()

# Define limit to control how many rows to process
limit = 100  # Modify the limit as needed

# Loop through each row in the DataFrame
for i in range(len(final_vsp)):
    ans = final_vsp.iloc[i]['answer']
    index = ans.find("CVSS:3.1")  # Find the starting index of the CVSS vector
    if index != -1:  # Ensure the vector is found
        vector = ans[index:index+len(example)]  # Extract the vector
    else:
        vector = None  # Handle cases where no CVSS vector is found
    new_data['vector'].append(vector)

# Create a new DataFrame for the extracted vectors
new_df = pd.DataFrame(new_data, index=final_vsp.index[:limit])

# Select the old data (first 'limit' rows)
old_df = final_vsp.head(limit)

# Merge the old data with the new CVSS vectors
final_vspllama = pd.merge(old_df, new_df, left_index=True, right_index=True)

# Save the merged DataFrame to a new CSV file
final_vspllama.to_csv('ourDatasets/responses/vsp/final_vspL_vector.csv', index=False)

print("CSV file 'final_vspL_vector' has been created.")


CSV file 'final_vspL_vector' has been created.


In [12]:
def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    cvss_score = c.scores()[0]
    return cvss_score

def compute_vsp_madlama(fname, col):
    cvss_prefix = ''   # should be empty string if the model responds with the prefix
    #df = pd.read_csv(fname, sep='\t')
    df = final_vspllama
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_vector = cvss_prefix + pred
            pred_score = get_cvss_score(pred_vector)
            gt_score = get_cvss_score(gt)
            print(pred_score, gt_score)
            print(abs(pred_score-gt_score))
            error += abs(pred_score-gt_score)
        except Exception as e:
            print('Invalid response at row {}'.format(idx+1))
            print(e)
            continue
        total += 1
    print(error)
    return error/total

In [13]:
compute_vsp_madlama("", col = "vector")

9.9 7.8
2.1000000000000005
10.0 8.4
1.5999999999999996
7.5 6.6
0.9000000000000004
9.1 6.1
3.0
9.9 8.8
1.0999999999999996
6.8 5.5
1.2999999999999998
10.0 9.8
0.1999999999999993
9.8 8.1
1.700000000000001
8.1 7.8
0.2999999999999998
10.0 9.8
0.1999999999999993
6.8 5.5
1.2999999999999998
10.0 4.6
5.4
7.5 7.5
0.0
7.1 8.8
1.700000000000001
6.5 8.8
2.3000000000000007
7.6 7.1
0.5
5.4 5.4
0.0
10.0 7.8
2.2
9.8 5.5
4.300000000000001
10.0 8.8
1.1999999999999993
5.4 8.8
3.4000000000000004
9.8 6.5
3.3000000000000007
10.0 8.8
1.1999999999999993
8.8 7.8
1.0000000000000009
8.1 7.8
0.2999999999999998
9.8 5.5
4.300000000000001
9.8 7.5
2.3000000000000007
7.5 4.3
3.2
7.3 5.4
1.8999999999999995
10.0 6.5
3.5
6.7 7.8
1.0999999999999996
10.0 7.1
2.9000000000000004
7.5 5.5
2.0
8.3 5.4
2.9000000000000004
9.3 6.5
2.8000000000000007
7.7 6.3
1.4000000000000004
10.0 7.0
3.0
8.3 6.1
2.200000000000001
8.2 5.3
2.8999999999999995
5.4 7.1
1.6999999999999993
8.5 6.5
2.0
8.6 9.8
1.200000000000001
8.6 6.1
2.5
9.8 6.2
3.60000

2.0494949494949495

: 

## generation of the summary 

In [5]:
import pandas as pd

# Chargement des fichiers d'évaluation
mcq_llama = pd.read_csv('ourDatasets/responses/mcq/final_mcq.csv')
mcq_mistral = pd.read_csv('ourDatasets/responses/mcq/final_mcq_mis.csv')

rcm_llama = pd.read_csv('ourdatasets/responses/rcm/final_rcm_extracted_codelama.csv')
rcm_mistral = pd.read_csv('ourdatasets/responses/rcm/final_rcm_extracted_codeM.csv')

vsp_llama = pd.read_csv('ourDatasets/responses/vsp/final_vspL_vector.csv')
vsp_mistral = pd.read_csv('ourDatasets/responses/vsp/final_vspM_Vector.csv')

# Fonction evaluate
def evaluate(df):
    correct_predictions = 0
    for i in range(len(df)):
        gt = df.iloc[i]['GT']
        llm_answer = df.iloc[i]['answer']
        predicted = extract_predicted_option(llm_answer)
        if predicted == gt:
            correct_predictions += 1
    total_predictions = len(df)
    return correct_predictions / total_predictions * 100

# Fonction extract_predicted_option
def extract_predicted_option(s: str):
    options = ['A', 'B', 'C', 'D']
    s = s[::-1]  # Inverse la chaîne pour trouver la première lettre
    for i in range(len(s)):
        if s[i] in options and (i == 0 or not s[i - 1].isalpha()) and (i == len(s) - 1 or not s[i + 1].isalpha()):
            return s[i]
    return ""

# Calcul des scores pour chaque évaluation
mcq_llama_accuracy = evaluate(mcq_llama)
mcq_mistral_accuracy = evaluate(mcq_mistral)

rcm_llama_accuracy = (rcm_llama['code'] == rcm_llama['GT']).mean() * 100
rcm_mistral_accuracy = (rcm_mistral['code'] == rcm_mistral['GT']).mean() * 100

# Fonction get_cvss_score (déjà définie dans votre code)
from cvss import CVSS3
def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    return c.scores()[0]

# Calcul des MAD pour VSP
def compute_vsp_madlama(fname, col):
    df = vsp_llama
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_score = get_cvss_score(pred)
            gt_score = get_cvss_score(gt)
            error += abs(pred_score - gt_score)
        except Exception as e:
            print(f'Invalid response at row {idx+1}: {e}')
            continue
        total += 1
    return error / total

def compute_vsp_mad(fname, col):
    df = vsp_mistral
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row[col].upper()
        gt = row['GT'].upper()
        try:
            pred_score = get_cvss_score(pred)
            gt_score = get_cvss_score(gt)
            error += abs(pred_score - gt_score)
        except Exception as e:
            print(f'Invalid response at row {idx+1}: {e}')
            continue
        total += 1
    return error / total

vsp_llama_mad = compute_vsp_madlama("", col="vector")
vsp_mistral_mad = compute_vsp_mad("", col="vector")

# Création d'un DataFrame pour résumer les résultats
results = {
    "Model": ["LLaMA", "Mistral"],
    "MCQ Accuracy (%)": [mcq_llama_accuracy, mcq_mistral_accuracy],
    "RCM Accuracy (%)": [rcm_llama_accuracy, rcm_mistral_accuracy],
    "VSP MAD (Mean Absolute Difference)": [vsp_llama_mad, vsp_mistral_mad]
}

summary_df = pd.DataFrame(results)

# Sauvegarde des résultats dans un fichier CSV
summary_df.to_csv('evaluation_summary.csv', index=False)

print("Les résultats des évaluations ont été enregistrés dans 'evaluation_summary.csv'.")
print(summary_df)


Invalid response at row 62: Unknown value "M" in field "A:M"
Invalid response at row 3: Unknown value "A" in field "PR:A"
Invalid response at row 112: Unknown value "R" in field "AV:R"
Les résultats des évaluations ont été enregistrés dans 'evaluation_summary.csv'.
     Model  MCQ Accuracy (%)  RCM Accuracy (%)  \
0    LLaMA             40.95         58.666667   
1  Mistral             44.45         50.285714   

   VSP MAD (Mean Absolute Difference)  
0                            2.049495  
1                            2.416216  
